In [1]:
import pandas as pd
import gurobipy as gp
from gurobipy import GRB

In [2]:
data = [[ 0.5, None, None, None, None, None],
        [ 0.5,  0.3, None, None, None, None],
        [ 1.0,  0.5, None, None, None, None],
        [ 0.2,  0.2, None, None, None, None],
        [ 1.0,  2.0, None,  1.0, None, None],
        [None, None,  1.5,  0.5,  1.5, None],
        [ 1.5, None,  0.5,  0.5,  2.0,  0.2],
        [ 2.0, None,  1.5,  1.0, None,  1.5],
        [None, None, None,  0.5,  0.5,  0.5],
        [ 1.0, None,  1.0, None,  1.5,  1.5]]

data2 = [[ 0.5, None, None, None, None, None],
         [ 0.5,  0.3, None, None, None, None],
         [ 1.0,  0.5, None, None, None, None],
         [ 0.2,  0.2, None, None, None, None],
         [ 1.0,  2.0, None,  1.0, None, None],
         [None, None,  1.5,  0.5,  1.5, None],
         [ 1.5, None,  0.5,  0.5,  2.0,  0.2],
         [ 2.0, None,  1.5,  1.0, None,  1.5],
         [None, None, None,  0.5,  0.5,  0.5],
         [ 1.0, None,  1.0, None,  1.5,  1.5]]

CUSTOMERS = ['C1', 'C2', 'C3', 'C4', 'C5', 'C6']
FACTORIES = ['LIVERPOOL', 'BRIGHTON'] 
DEPOTS    = ['NEWCASTLE', 'BIRMINGHAM', 'LONDON', 'EXERTER']

supply_cost = pd.DataFrame(data, columns=FACTORIES+DEPOTS, index=DEPOTS+CUSTOMERS).fillna(0)
poss_supply = {i: [j for j in DEPOTS+CUSTOMERS if supply_cost[i][j] != 0 ] for i in FACTORIES+DEPOTS}
poss_demand = {j: [i for i in FACTORIES+DEPOTS if supply_cost[i][j] != 0 ] for j in DEPOTS+CUSTOMERS}
connections = [(i,j) for i in poss_supply.keys() for j in poss_supply[i]]

capacity  = dict(zip(FACTORIES + DEPOTS, [150000, 200000, 70000, 50000, 100000, 40000]))
requirem  = dict(zip(CUSTOMERS, [50000, 10000, 40000, 35000, 60000, 20000]))

### Case 1: No prefered suppliers

In [3]:
model = gp.Model('10 Distribution 1 case1')

# add vars
supplyto = model.addVars(connections,
                         name='supplyto')

# objective function
model.setObjective(gp.quicksum(supply_cost[i][j]*supplyto[i,j] for i, j in connections))

# add constraints
model.addConstrs((gp.quicksum(supplyto[i,j] for j in poss_supply[i]) <= capacity[i] for i in FACTORIES+DEPOTS),
                 name='capacity')
model.addConstrs((gp.quicksum(supplyto[i,j] for i in poss_demand[j]) == gp.quicksum(supplyto[j,k] for k in poss_supply[j]) for j in DEPOTS),
                 name='depot_cap')                      # into depot == out of depot
model.addConstrs((gp.quicksum(supplyto[i,j] for i in poss_demand[j]) >= requirem[j] for j in CUSTOMERS),
                 name='requirement')

model.update()

Using license file C:\Users\Nara\gurobi.lic
Academic license - for non-commercial use only - expires 2021-03-12


In [4]:
model.write('10 Distribution 1 case1.lp')
model.optimize()

Gurobi Optimizer version 9.1.0 build v9.1.0rc0 (win64)
Thread count: 4 physical cores, 8 logical processors, using up to 8 threads
Optimize a model with 16 rows, 29 columns and 75 nonzeros
Model fingerprint: 0x719b6be4
Coefficient statistics:
  Matrix range     [1e+00, 1e+00]
  Objective range  [2e-01, 2e+00]
  Bounds range     [0e+00, 0e+00]
  RHS range        [1e+04, 2e+05]
Presolve removed 1 rows and 1 columns
Presolve time: 0.01s
Presolved: 15 rows, 28 columns, 73 nonzeros

Iteration    Objective       Primal Inf.    Dual Inf.      Time
       0    0.0000000e+00   2.687500e+04   0.000000e+00      0s
      11    1.9850000e+05   0.000000e+00   0.000000e+00      0s

Solved in 11 iterations and 0.02 seconds
Optimal objective  1.985000000e+05


In [5]:
print('Distribution cost of ${:.2f}'.format(model.objVal))

dist_solution = pd.DataFrame([], columns=FACTORIES+DEPOTS, index=DEPOTS+CUSTOMERS).fillna(0)
for i, t in supplyto.keys():
    dist_solution[i][t] = supplyto[i,t].x
print('===SELLING PLAN')
print(dist_solution)

Distribution cost of $198500.00
===SELLING PLAN
            LIVERPOOL  BRIGHTON  NEWCASTLE  BIRMINGHAM  LONDON  EXERTER
NEWCASTLE           0         0          0           0       0        0
BIRMINGHAM          0     50000          0           0       0        0
LONDON              0     55000          0           0       0        0
EXERTER         40000         0          0           0       0        0
C1              50000         0          0           0       0        0
C2                  0         0          0       10000       0        0
C3                  0         0          0           0       0    40000
C4                  0         0          0       35000       0        0
C5                  0         0          0        5000   55000        0
C6              20000         0          0           0       0        0


In [6]:
print('First, which depots are operating at full capacity?')
for d in DEPOTS:
    if sum([supplyto[d,j].x for j in poss_supply[d]]) == capacity[d]:
        print(d)


print('\nShadow Prices of all Depots')
for const in model.getConstrs():
    if const.PI > 0 and const.ConstrName[:1] == 'd':
        print(const.ConstrName, const.PI)
        
print('\n Check the shadow prices of depots operating at full capacity')

First, which depots are operating at full capacity?
BIRMINGHAM
EXERTER

Shadow Prices of all Depots
depot_cap[NEWCASTLE] 0.5
depot_cap[BIRMINGHAM] 0.3
depot_cap[LONDON] 0.5
depot_cap[EXERTER] 0.2

 Check the shadow prices of depots operating at full capacity


In [ ]:
print('Time to Sensitivity Analysis')
# eu nao sei muito bem como fazer esse relatorio bem bonitinho
#https://support.gurobi.com/hc/en-us/community/posts/360063478872-How-to-generate-sensitivity-analysis-report-in-detail

for i, j in supplyto.keys():
    print('{} to {}: {}'.format(i,j,supplyto[i,j].RC))
    print('{} to {}: {}'.format(i,j,supplyto[i,j].SAObjUp))
    print('{} to {}: {}'.format(i,j,supplyto[i,j].SAObjLow))

### Case 2: With preferred suppliers

In [7]:
# costs table, but customers can only receive from their preferred suppliers
data2 = [[ 0.5, None, None, None, None, None],
         [ 0.5,  0.3, None, None, None, None],
         [ 1.0,  0.5, None, None, None, None],
         [ 0.2,  0.2, None, None, None, None],
         [ 1.0, None, None, None, None, None],
         [None, None,  1.5, None, None, None],
         [ 1.5, None,  0.5,  0.5,  2.0,  0.2],
         [ 2.0, None,  1.5,  1.0, None,  1.5],
         [None, None, None,  0.5, None, None],
         [None, None, None, None,  1.5,  1.5]]

# then, we rebuild the dictionaries based on data2
supply_cost2 = pd.DataFrame(data2, columns=FACTORIES+DEPOTS, index=DEPOTS+CUSTOMERS).fillna(0)
poss_supply2 = {i: [j for j in DEPOTS+CUSTOMERS if supply_cost2[i][j] != 0 ] for i in FACTORIES+DEPOTS}
poss_demand2 = {j: [i for i in FACTORIES+DEPOTS if supply_cost2[i][j] != 0 ] for j in DEPOTS+CUSTOMERS}
connections2 = [(i,j) for i in poss_supply2.keys() for j in poss_supply2[i]]

In [10]:
model2 = gp.Model('10 Distribution 1 case2')

# add vars
supplyto2 = model2.addVars(connections,
                         name='supplyto')

# objective function
model2.setObjective(gp.quicksum(supply_cost[i][j]*supplyto2[i,j] for i, j in connections))

# add constraints
model2.addConstrs((gp.quicksum(supplyto2[i,j] for j in poss_supply2[i]) <= capacity[i] for i in FACTORIES+DEPOTS),
                 name='capacity')
model2.addConstrs((gp.quicksum(supplyto2[i,j] for i in poss_demand2[j]) == gp.quicksum(supplyto2[j,k] for k in poss_supply2[j]) for j in DEPOTS),
                 name='depot_cap')                        # into depot == out of depot
model2.addConstrs((gp.quicksum(supplyto2[i,j] for i in poss_demand2[j]) >= requirem[j] for j in CUSTOMERS),
                 name='requirement')

model2.update()

In [11]:
model2.write('10 Distribution 1 case2.lp')
model2.optimize()

Gurobi Optimizer version 9.1.0 build v9.1.0rc0 (win64)
Thread count: 4 physical cores, 8 logical processors, using up to 8 threads
Optimize a model with 16 rows, 29 columns and 53 nonzeros
Model fingerprint: 0xf91f3224
Coefficient statistics:
  Matrix range     [1e+00, 1e+00]
  Objective range  [2e-01, 2e+00]
  Bounds range     [0e+00, 0e+00]
  RHS range        [1e+04, 2e+05]
Presolve removed 3 rows and 9 columns
Presolve time: 0.01s

Solved in 0 iterations and 0.01 seconds
Infeasible model


Not possible to fulfill all customers preferences

### Case 3: Maximize customers preferences
It's not possible to satisfy all customers preferences, so let's try to maximize it, by finding a way where customers receive from their preferred suplliers as much as possible.

In [38]:
pref_supp = dict(zip(CUSTOMERS, [['LIVERPOOL'], ['NEWCASTLE'], [], [], ['BIRMINGHAM'], ['LONDON', 'EXERTER']]))
w_pref = [(j,i) for i in pref_supp.keys() for j in pref_supp[i]]

In [42]:
model3 = gp.Model('10 Distribution 1 case3')

# add vars
supplyto3 = model3.addVars(connections,
                         name='supplyto')

# objective function
model3.setObjectiveN(-(gp.quicksum(supplyto3[i,j] for i, j in w_pref)), 1)
model3.setObjectiveN((gp.quicksum(supply_cost[i][j]*supplyto3[i,j] for i, j in connections)), 0)

# add constraints
model3.addConstrs((gp.quicksum(supplyto3[i,j] for j in poss_supply[i]) <= capacity[i] for i in FACTORIES+DEPOTS),
                 name='capacity')
model3.addConstrs((gp.quicksum(supplyto3[i,j] for i in poss_demand[j]) == gp.quicksum(supplyto3[j,k] for k in poss_supply[j]) for j in DEPOTS),
                 name='depot_cap')                      # into depot == out of depot
model3.addConstrs((gp.quicksum(supplyto3[i,j] for i in poss_demand[j]) >= requirem[j] for j in CUSTOMERS),
                 name='requirement')

model3.update()

In [43]:
model3.write('10 Distribution 1 case3.lp')
model3.optimize()

Gurobi Optimizer version 9.1.0 build v9.1.0rc0 (win64)
Thread count: 4 physical cores, 8 logical processors, using up to 8 threads
Optimize a model with 16 rows, 29 columns and 75 nonzeros
Model fingerprint: 0x10f3bd78
Variable types: 29 continuous, 0 integer (0 binary)
Coefficient statistics:
  Matrix range     [1e+00, 1e+00]
  Objective range  [2e-01, 2e+00]
  Bounds range     [0e+00, 0e+00]
  RHS range        [1e+04, 2e+05]

---------------------------------------------------------------------------
Multi-objectives: starting optimization with 2 objectives (1 combined) ...
---------------------------------------------------------------------------
---------------------------------------------------------------------------

Multi-objectives: optimize objective 1 (weighted) ...
---------------------------------------------------------------------------

Optimize a model with 16 rows, 29 columns and 75 nonzeros
Coefficient statistics:
  Matrix range     [1e+00, 1e+00]
  Objective range

In [45]:
model3.params.ObjNumber = 0
print('Distribution cost of ${:.2f}'.format(model3.ObjNVal))

dist_solution = pd.DataFrame([], columns=FACTORIES+DEPOTS, index=DEPOTS+CUSTOMERS).fillna(0)
for i, j in supplyto3.keys():
    dist_solution[i][j] = supplyto3[i,j].x
print('===DISTRIBUTION PLAN')
print(dist_solution)

Distribution cost of $261000.00
===DISTRIBUTION PLAN
            LIVERPOOL  BRIGHTON  NEWCASTLE  BIRMINGHAM  LONDON  EXERTER
NEWCASTLE       10000         0          0           0       0        0
BIRMINGHAM          0     50000          0           0       0        0
LONDON              0     30000          0           0       0        0
EXERTER         40000         0          0           0       0        0
C1              65000         0          0           0       0        0
C2                  0         0      10000           0       0        0
C3                  0         0          0           0       0    40000
C4              35000         0          0           0       0        0
C5                  0         0          0       50000   10000        0
C6                  0         0          0           0   20000        0


This solution is not the same as the book's. And probably not the "minimum".